# Linear Programming

### Task 01

There are 168 hours in a week. Say we want to allocate our time between classes and
studying (S), fun activities and going to parties (P), and everything else (E) (eating, sleeping,
taking showers, etc).

Suppose that to survive we need to spend at least 56 hours on E (8
hours/day). To maintain sanity we need P + E ≥ 70. To pass our courses, we need S ≥ 60,
but more if don’t sleep enough or spend too much time partying: 2S + E − 3P ≥ 150. (E.g.,
if don’t go to parties at all then this isn’t a problem, but if we spend more time on P then
need to sleep more or study more).

- Variables: S, P, E
- Objective: maximize 2P + E
- Subject to сonstraints: 
 * S + P + E = 168
 * E ≥ 56
 *  S ≥ 60
 *  2S + E − 3P ≥ 150
 *  P + E ≥ 70
 *  P ≥ 0 (can’t spend negative time partying)

In [7]:
from scipy.optimize import linprog

c = [-2, 0, -1]
Au = [[0, 0, -1], [-1, 0, 0], [-2, 3, -1], [0, -1, -1], [0, -1, 0]]
bu = [-56, -60, -150, -70, 0]
Ae = [[1, 1, 1]]
be = [[168]]
res = linprog(c, A_ub=Au, b_ub=bu, A_eq = Ae, b_eq = be, method='simplex', options={"disp": True})

Optimization terminated successfully.
         Current function value: -266.000000 
         Iterations: 8
